In [1]:
# %pip install plotly
# %pip install nbformat
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

os.getcwd()

'/Users/aneeshussain/Code/diss/analysis'

In [2]:
# Get the current directory
current_dir = os.getcwd()

# Change directory to the parent directory (diss)
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)

# Append the parent directory to the system path (optional, if you want to add it to the import path)
sys.path.append(parent_dir)

# Print the new current working directory to confirm the change
print(f"Changed directory to: {os.getcwd()}")

Changed directory to: /Users/aneeshussain/Code/diss


In [3]:

filepaths = {
    'binance': './data/processed/binance.csv',
    'okx': './data/processed/okx.csv',
    'bybit': './data/processed/bybit.csv'
}

# List of specified pairs
specified_pairs = ['BTCUSDCM', 'BTCUSDT', 'ETHUSDCM', 'ETHUSDT']

# Initialize a dictionary to store counts for each exchange
exchange_funding_stats = {}

# Loop through each exchange
for exchange, filepath in filepaths.items():
    # Load the data for the current exchange
    df = pd.read_csv(filepath)

    # Convert 'funding rate' to float
    df['funding rate'] = pd.to_numeric(df['funding rate'], errors='coerce')

    # Filter the DataFrame to only include the specified pairs
    filtered_df = df[df['pair'].isin(specified_pairs)]

    # Initialize dictionaries to store counts for this exchange
    positive_funding_counts = {}
    total_funding_counts = {}

    # Iterate through each specified pair
    for pair in specified_pairs:
        # Filter rows for the current pair
        pair_df = filtered_df[filtered_df['pair'] == pair]

        # Count positive funding rates
        positive_count = (pair_df['funding rate'] > 0).sum()
        total_count = len(pair_df)

        # Store the counts in the dictionaries
        positive_funding_counts[pair] = positive_count
        total_funding_counts[pair] = total_count

    # Create a DataFrame for this exchange to display the counts
    funding_stats = pd.DataFrame({
        'positive_count': positive_funding_counts,
        'total_count': total_funding_counts
    })

    # Calculate the percentage of positive funding rates
    funding_stats['positive_percentage'] = (funding_stats['positive_count'] / funding_stats['total_count']) * 100

    # Store the result for this exchange
    exchange_funding_stats[exchange] = funding_stats

# Display the results for each exchange
for exchange, stats in exchange_funding_stats.items():
    print(f"{exchange} Funding stats")
    print(stats)
    print("\n")

binance Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1435         1642            87.393423
BTCUSDT             1517         1642            92.387333
ETHUSDCM            1467         1642            89.342266
ETHUSDT             1539         1642            93.727162


okx Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1132         1642            68.940317
BTCUSDT             1203         1642            73.264312
ETHUSDCM            1163         1642            70.828258
ETHUSDT             1226         1642            74.665043


bybit Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1472         1642            89.646772
BTCUSDT             1494         1642            90.986602
ETHUSDCM            1481         1642            90.194884
ETHUSDT             1474         1642            89.768575




In [80]:
trades_filepath = './results/simple_threshold_open/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        1128
bybit       353
binance     344
Name: count, dtype: int64


In [19]:
hstr = pd.read_csv(f'./results/hold/collateral_log.csv')
ststr = pd.read_csv(f'./results/simple_threshold/collateral_log.csv')
srstr = pd.read_csv(f'./results/simple_reinvest/collateral_log.csv')
ctstr = pd.read_csv(f'./results/complex_threshold/collateral_log.csv')
crstr = pd.read_csv(f'./results/complex_reinvest/collateral_log.csv')

initial_investment = 10000000

hstr['time'] = pd.to_datetime(hstr['time'])
hstr.set_index('time', inplace=True)

ststr['time'] = pd.to_datetime(ststr['time'])
ststr.set_index('time', inplace=True)

srstr['time'] = pd.to_datetime(srstr['time'])
srstr.set_index('time', inplace=True)

ctstr['time'] = pd.to_datetime(ctstr['time'])
ctstr.set_index('time', inplace=True)

crstr['time'] = pd.to_datetime(crstr['time'])
crstr.set_index('time', inplace=True)

collateral_columns = [
    'binance_btc_collateral', 'binance_eth_collateral', 'binance_liquid_cash',
    'okx_btc_collateral', 'okx_eth_collateral', 'okx_liquid_cash',
    'bybit_btc_collateral', 'bybit_eth_collateral', 'bybit_liquid_cash'
]

funding_columns = ['binance_funding', 'okx_funding', 'bybit_funding']

unrealised_pnl_columns = ['binance_unrealised_pnl', 'okx_unrealised_pnl', 'bybit_unrealised_pnl']

hstr['pnl_basis'] = hstr[collateral_columns].sum(axis=1) - initial_investment
hstr['pnl_funding'] = hstr[funding_columns].sum(axis=1)
hstr['total_pnl'] = hstr['pnl_basis'] + hstr['pnl_funding']

hstr['cumulative_total_pnl_pct'] = (hstr['total_pnl'] / initial_investment) * 100


ststr['pnl_basis'] = ststr[collateral_columns].sum(axis=1) - initial_investment
ststr['pnl_funding'] = ststr[funding_columns].sum(axis=1)
ststr['total_pnl'] = ststr['pnl_basis'] + ststr['pnl_funding']

ststr['cumulative_total_pnl_pct'] = (ststr['total_pnl'] / initial_investment) * 100


srstr['pnl_basis'] = srstr[collateral_columns].sum(axis=1) - initial_investment
srstr['pnl_funding'] = srstr[funding_columns].sum(axis=1)
srstr['total_pnl'] = srstr['pnl_basis'] + srstr['pnl_funding']

srstr['cumulative_total_pnl_pct'] = (srstr['total_pnl'] / initial_investment) * 100


ctstr['pnl_basis'] = ctstr[collateral_columns].sum(axis=1) - initial_investment
ctstr['pnl_funding'] = ctstr[funding_columns].sum(axis=1)
ctstr['total_pnl'] = ctstr['pnl_basis'] + ctstr['pnl_funding']

ctstr['cumulative_total_pnl_pct'] = (ctstr['total_pnl'] / initial_investment) * 100


crstr['pnl_basis'] = crstr[collateral_columns].sum(axis=1) - initial_investment
crstr['pnl_funding'] = crstr[funding_columns].sum(axis=1)
crstr['total_pnl'] = crstr['pnl_basis'] + crstr['pnl_funding']

crstr['cumulative_total_pnl_pct'] = (crstr['total_pnl'] / initial_investment) * 100

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=hstr.index,
    y=hstr['total_pnl'],
    mode='lines',
    name='Buy & Hold',
    line=dict(width=1, color='blue'),
    hovertemplate='Hold P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=hstr.index,
    y=hstr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Buy & Hold',
    line=dict(width=1, color='blue'),
    hovertemplate='Hold P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=ststr.index,
    y=ststr['total_pnl'],
    mode='lines',
    name='Simple',
    line=dict(width=1, color='orange'),
    hovertemplate='Simple Threshold P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=ststr.index,
    y=ststr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Simple',
    line=dict(width=1, color='orange'),
    hovertemplate='Simple Threshold P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=srstr.index,
    y=srstr['total_pnl'],
    mode='lines',
    name='Simple Reinvest',
    line=dict(width=1, color='green'),
    hovertemplate='Simple Reinvest P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=srstr.index,
    y=srstr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Simple Reinvest',
    line=dict(width=1, color='green'),
    hovertemplate='Simple Reinvest P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=ctstr.index,
    y=ctstr['total_pnl'],
    mode='lines',
    name='Complex',
    line=dict(width=1, color='purple'),
    hovertemplate='Complex Threshold P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=ctstr.index,
    y=ctstr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Complex',
    line=dict(width=1, color='purple'),
    hovertemplate='Complex Threshold P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=crstr.index,
    y=crstr['total_pnl'],
    mode='lines',
    name='Complex Reinvest',
    line=dict(width=1, color='red'),
    hovertemplate='Complex Reinvest P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=crstr.index,
    y=crstr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Complex Reinvest',
    line=dict(width=1, color='red'),
    hovertemplate='Complex Reinvest P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.update_layout(
    title=f'Cumulative P&L Over Time',
    yaxis=dict(title='P&L ($)'),
    yaxis2=dict(title='P&L (%)', overlaying='y', side='right'),
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.2,
        xanchor='center',
        yanchor='top'
    ),
    width=800,
    height=600,
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.update_xaxes(
    tickformat='%b %Y',
    tickangle=0,
    showgrid=True,
    zeroline=True,
    showticklabels=True,
    dtick='M3'
)

fig.show()

# fig.write_image('./results/compare_cumulative_pnl.png')

In [7]:
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file
file_path = './data/all_exchanges.csv' 
df = pd.read_csv(file_path)

# Filter for perpetual contracts only
perp_df = df[df['contract'] == 'perpetual']

# Convert 'time' to datetime format
perp_df['time'] = pd.to_datetime(perp_df['time'])

# Group by time and sum the volume
daily_volume = perp_df.groupby('time')['volume'].sum().reset_index()

# Calculate the 30-day rolling average for daily volume
daily_volume['30_day_rolling_avg'] = daily_volume['volume'].rolling(window=30).mean()

# Plotting
fig = go.Figure()

# Plot the actual daily volume
fig.add_trace(go.Scatter(
    x=daily_volume['time'],
    y=daily_volume['volume'],
    mode='lines',
    name='Total Daily Volume',
    line=dict(width=2)
))

# Plot the 30-day rolling average
fig.add_trace(go.Scatter(
    x=daily_volume['time'],
    y=daily_volume['30_day_rolling_avg'],
    mode='lines',
    name='30-Day Rolling Average',
    line=dict(width=2, color='orange', dash='dash')  # Different style for the rolling average
))

fig.update_layout(
    title="Total Daily Trading Volume for Perpetual Contracts with 30-Day Rolling Average",
    xaxis_title="Time",
    yaxis_title="Total Trading Volume",
    template='plotly_white',
    hovermode='x unified',
    width=900,
    height=600,
    legend=dict(orientation='h', x=0.5, y=-0.2, xanchor='center')
)

fig.show()

/var/folders/mf/b67_blxs1rj380h3rh2kgtlh0000gn/T/ipykernel_71737/1841664186.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file
file_path = './data/all_exchanges.csv' 
df = pd.read_csv(file_path)

# Filter for perpetual contracts
perp_df = df[df['contract'] == 'perpetual']

# Convert 'time' to datetime format
perp_df['time'] = pd.to_datetime(perp_df['time'])

# Group by time and sum the volume for perpetual contracts
daily_volume_perp = perp_df.groupby('time')['volume'].sum().reset_index()

# Calculate the 30-day rolling average for perpetual contracts
daily_volume_perp['30_day_rolling_avg'] = daily_volume_perp['volume'].rolling(window=30).mean()

# Filter for spot contracts
spot_df = df[df['contract'] == 'spot']

# Group by time and sum the volume for spot contracts
daily_volume_spot = spot_df.groupby('time')['volume'].sum().reset_index()

# Calculate the 30-day rolling average for spot contracts
daily_volume_spot['30_day_rolling_avg'] = daily_volume_spot['volume'].rolling(window=30).mean()

# Plotting
fig = go.Figure()

# # Plot the actual daily volume for perpetual contracts
# fig.add_trace(go.Scatter(
#     x=daily_volume_perp['time'],
#     y=daily_volume_perp['volume'],
#     mode='lines',
#     name='Perpetual Daily Volume',
#     line=dict(width=2, color='blue')
# ))

# Plot the 30-day rolling average for perpetual contracts
fig.add_trace(go.Scatter(
    x=daily_volume_perp['time'],
    y=daily_volume_perp['30_day_rolling_avg'],
    mode='lines',
    name='Perpetual 30-Day Rolling Average',
    line=dict(width=2, color='blue')  # Different style for the rolling average
))

# # Plot the actual daily volume for spot contracts
# fig.add_trace(go.Scatter(
#     x=daily_volume_spot['time'],
#     y=daily_volume_spot['volume'],
#     mode='lines',
#     name='Spot Daily Volume',
#     line=dict(width=2, color='green')
# ))

# Plot the 30-day rolling average for spot contracts
fig.add_trace(go.Scatter(
    x=daily_volume_spot['time'],
    y=daily_volume_spot['30_day_rolling_avg'],
    mode='lines',
    name='Spot 30-Day Rolling Average',
    line=dict(width=2, color='red')  # Different style for the rolling average
))

fig.update_layout(
    title="Total Daily Trading Volume for Perpetual and Spot Contracts with 30-Day Rolling Average",
    xaxis_title="Time",
    yaxis_title="Total Trading Volume",
    template='plotly_white',
    hovermode='x unified',
    width=900,
    height=600,
    legend=dict(orientation='h', x=0.5, y=-0.2, xanchor='center')
)

fig.update_xaxes(
    tickformat='%b %Y',
    tickangle=0,
    showgrid=True,
    zeroline=True,
    showticklabels=True,
    dtick='M3'
)

fig.show()

/var/folders/mf/b67_blxs1rj380h3rh2kgtlh0000gn/T/ipykernel_71737/2059318021.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
import pandas as pd

df = pd.read_csv('./results/hold/yield.csv')
print(f"Hold average funding rate: {df['total_yield'].mean()}")

df = pd.read_csv('./results/simple_threshold/yield.csv')
print(f"Simple average funding rate: {df['total_yield'].mean()}")

df = pd.read_csv('./results/simple_reinvest/yield.csv')
print(f"Simple Reinvest average funding rate: {df['total_yield'].mean()}")

df = pd.read_csv('./results/complex_threshold/yield.csv')
print(f"Complex average funding rate: {df['total_yield'].mean()}")

df = pd.read_csv('./results/complex_reinvest/yield.csv')
print(f"Complex Reinvest average funding rate: {df['total_yield'].mean()}")

Hold average funding rate: 6.15502785928074
Simple average funding rate: 1.4530226158881023
Simple Reinvest average funding rate: 1.4275916506830721
Complex average funding rate: 1.319335573847867
Complex Reinvest average funding rate: 1.2910481275910461
